In [1]:
import numpy as np
import pandas as pd

def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen

def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen) 
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
        #season_averages.append(1)
        #print(season_averages[0])
    #print('n_seasons: ', n_seasons, ' season_averages len: ', len(season_averages))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            #print(series.compute().iloc[0])
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals

def triple_exponential_smoothing(series, slen=4, alpha=0.716, beta=0.029, gamma=0.993, n_preds=1):
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result

In [ ]:
import pandas as pd
import multiprocessing as mp

pool = mp.Pool(4) # use 4 processes

data2 = pd.read_csv('User1.csv', index_col=0)
data2['SPENT'] = 0
#data = pd.read_csv('c:/Users/dani/PycharmProjects/WOW/CSV/TCUSTCOUPONITEMSSPENT.csv', index_col=None).reset_index(drop=True)
data = pd.read_csv('User10869649.csv', index_col=0).reset_index(drop=True)

def f(group):
   df = pd.merge(left=group.reset_index(), right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
   df['SPENT2'] = df.SPENT_x + df.SPENT_y
   df.SPENT2 = df.SPENT2.fillna(0)
   df['DATE'] = 100*df.YEARIND + df.WEEKIND
   df.sort_values(['DATE'], inplace=True)
   df = df.fillna(method='ffill')
   df = df.dropna()
   df = df.reset_index()
   hws = triple_exponential_smoothing(df.SPENT2)
   return hws[-1] # return the predicted value

def process_frame(data_account):
    account = data_account.ACCOUNT_CODE.min()
    grouped = data_account.groupby(['DEAL_CODE', 'COUPON_CODE']).apply(f)
    grouped.to_csv('TCUST_predicted-' + str(account) + '.csv')    
    return len(df)

funclist = []
for account in data.ACCOUNT_CODE.unique():
   data_account = data.loc[data.ACCOUNT_CODE==account]
   # process each data frame
   f = pool.apply_async(process_frame,[data_account])
   funclist.append(f)

result = 0
for f in funclist:
    result += f.get(timeout=1000) # timeout in 1000 seconds

print ("There are %d rows of data"%result)